In [42]:
import sqlite3
import re

In [49]:
conn = sqlite3.connect('emaildb.sqlite')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS Counts')

cur.execute('''
CREATE TABLE Counts (org TEXT, count INTEGER)''')

file_name = input('Enter file name: ')
if len(file_name) < 1:
    file_name = 'mbox.txt'

try:
    with open(file_name, 'r') as file:
        for line in file:
            emails = re.findall(r'\b[\w\.-]+@[\w\.-]+\b', line)
            for email in emails:
                domain = email.split('@')[1]
                cur.execute('SELECT count FROM Counts WHERE org = ? ', (domain,))
                row = cur.fetchone()
                if row is None:
                    cur.execute('INSERT INTO Counts (org, count) VALUES (?, 1)', (domain,))
                else:
                    cur.execute('UPDATE Counts SET count = count + 1 WHERE org = ?', (domain,))
        conn.commit()

        # Retrieve and print the top 10 results
        sqlstr = 'SELECT org, count FROM Counts ORDER BY count DESC LIMIT 10'
        print('Counts:')
        for row in cur.execute(sqlstr):
            print(row[0], row[1])

except FileNotFoundError:
    print('File not found:', file_name)

cur.close()

Enter file name: mbox1.txt
Counts:
collab.sakaiproject.org 162
iupui.edu 38
umich.edu 29
nakamura.uits.iupui.edu 27
localhost 27
uct.ac.za 25
media.berkeley.edu 16
caret.cam.ac.uk 4
gmail.com 4
david-horwitz-6 4
